# Fussing to create a specdb compliable file

In [26]:
# imports
import h5py
import numpy as np
from astropy.table import Table

from specdb.build import utils as spbu
from specdb import defs as spb_defs

## Point to current hdf5

In [2]:
hdf = h5py.File('../../desi_dla_1.hdf', 'r')

In [3]:
list(hdf['z2.8-2.85'].keys())

['spec', 'zem']

In [4]:
zem = hdf['z2.8-2.85']['zem'].value

In [5]:
zem

array([2.82755788, 2.83039514, 2.84732377, ..., 2.84863594, 2.80509971,
       2.82674997])

## Let's do just one

In [6]:
hdfnew = h5py.File('z2.8_specdb_test.hdf', 'w')

In [7]:
group = 'z2.8-2.85'

In [8]:
_ = hdfnew.create_group(group)

### Start catalog

In [9]:
id_key = 'DESI_ID'

In [10]:
maindb, tkeys = spbu.start_maindb(id_key)

In [11]:
maindb.keys()

['RA', 'DEC', 'zem', 'sig_zem', 'flag_zem', 'flag_group', 'STYPE', 'DESI_ID']

In [12]:
gdict = {}

### Generate meta

In [13]:
meta = Table()

In [14]:
meta['zem_GROUP'] = hdf[group]['zem'].value

In [15]:
nqso = len(meta)
nqso

24143

#### All the others that are needed;  am filling in random junk for most

In [16]:
meta['RA_GROUP'] = np.random.uniform(0., 360., nqso)
meta['DEC_GROUP'] = np.random.uniform(-90, 90., nqso)
meta['EPOCH'] = 2000.
meta['sig_zem'] = 0.
meta['flag_zem'] = 'DESI'
meta['STYPE'] = 'QSO'
# Observation
meta['SPEC_FILE'] = 'filename'
meta['DATE-OBS'] = '2019-01-01'
#
meta['GROUP_ID'] = np.arange(len(meta)).astype(int)
# Spectrograph
meta['R'] = 3000.
meta['TELESCOPE'] = 'KPNO-4m'
meta['DISPERSER'] = 'ALL'
meta['INSTR'] = 'DESI'
meta['WV_MIN'] = 3800.  # Should be the right value
meta['WV_MAX'] = 9900.  # Should be the right value
meta['NPIX'] = 8000 # Should be the right value

#### Here are a few to more easily ID the sources -- Could take them from the mock file names

In [17]:
meta['PLATE'] = 1
meta['FIBERID'] = np.arange(len(meta)).astype(int) + 1

#### Check;  should only be missing DESI_ID

In [18]:
spbu.chk_meta(meta)

Missing ID key: DESI_ID


/home/xavier/local/Python/specdb/specdb/build/utils.py:298: UserWarning: unicode in column flag_zem.  Will convert to str for hdf5
  warnings.warn("unicode in column {:s}.  Will convert to str for hdf5".format(key))
/home/xavier/local/Python/specdb/specdb/build/utils.py:298: UserWarning: unicode in column STYPE.  Will convert to str for hdf5
  warnings.warn("unicode in column {:s}.  Will convert to str for hdf5".format(key))
/home/xavier/local/Python/specdb/specdb/build/utils.py:298: UserWarning: unicode in column SPEC_FILE.  Will convert to str for hdf5
  warnings.warn("unicode in column {:s}.  Will convert to str for hdf5".format(key))
/home/xavier/local/Python/specdb/specdb/build/utils.py:298: UserWarning: unicode in column DATE-OBS.  Will convert to str for hdf5
  warnings.warn("unicode in column {:s}.  Will convert to str for hdf5".format(key))
/home/xavier/local/Python/specdb/specdb/build/utils.py:298: UserWarning: unicode in column TELESCOPE.  Will convert to str for hdf5
  warn

False

### Group ID

In [19]:
flag_g = spbu.add_to_group_dict(group, gdict)
flag_g, gdict

(1, {'z2.8-2.85': 1})

### Catalog

In [20]:
# The output should be blank.  If not, your random RA/DEC accidentally overlapped
maindb = spbu.add_ids(maindb, meta, flag_g, tkeys, 'DESI_ID', first=(flag_g==1))

The following sources were previously in the DB
zem_GROUP RA_GROUP DEC_GROUP EPOCH ... DATE-OBS TELESCOPE DISPERSER INSTR
--------- -------- --------- ----- ... -------- --------- --------- -----


In [21]:
maindb

RA,DEC,zem,sig_zem,flag_zem,flag_group,STYPE,DESI_ID
float64,float64,float64,float64,bytes4,int64,bytes3,int64
296.419068958445,27.922570144990928,2.8275578835164197,0.0,DESI,1,QSO,0
26.705870509254012,-50.43326546303695,2.8303951445268467,0.0,DESI,1,QSO,1
274.55607580214354,-77.83190633858185,2.847323770984076,0.0,DESI,1,QSO,2
112.3664065846356,-37.356070138894815,2.8233004312496632,0.0,DESI,1,QSO,3
351.92722539355105,-79.6338013518388,2.813991705974331,0.0,DESI,1,QSO,4
109.85319746346963,-35.73617491534271,2.816572714364156,0.0,DESI,1,QSO,5
215.74009249321622,-39.016507348429315,2.802824530401267,0.0,DESI,1,QSO,6
170.31996121378265,-86.51471409995793,2.837784356321208,0.0,DESI,1,QSO,7
249.79843747219988,-79.97984789873612,2.810037763323635,0.0,DESI,1,QSO,8


### Check -- should be ok now

In [22]:
spbu.chk_meta(meta)

True

### Spectra

In [24]:
hdf.copy(group+'/spec', hdfnew[group])

### Meta

In [25]:
hdfnew[group]['meta'] = meta

## Finish

In [27]:
# redshift priority (needed but not used)
zpri = spb_defs.z_priority()

In [32]:
spbu.write_hdf(hdfnew, str('test_DESI'), maindb, zpri, gdict, str('v0.1'),
              Publisher='jding')

## Fussing

In [33]:
meta[0]

zem_GROUP,RA_GROUP,DEC_GROUP,EPOCH,sig_zem,GROUP_ID,R,WV_MIN,WV_MAX,NPIX,PLATE,FIBERID,flag_zem,STYPE,SPEC_FILE,DATE-OBS,TELESCOPE,DISPERSER,INSTR,DESI_ID
float64,float64,float64,float64,float64,int64,float64,float64,float64,int64,int64,int64,bytes4,bytes3,bytes8,bytes10,bytes7,bytes3,bytes4,int64
2.8275578835164197,296.419068958445,27.922570144990928,2000.0,0.0,0,3000.0,3800.0,9900.0,8000,1,1,DESI,QSO,filename,2019-01-01,KPNO-4m,ALL,DESI,0
